<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!pip install -qq transformers

In [59]:
import torch
import numpy as np
import pandas as pd

In [60]:
df_aug = pd.read_csv("11.csv", index_col='id')
df_aug

,tweet,subtask_a
id,,
864261.0,@USER She should ask a few native Americans wh...,OFF
864262.0,@USER She should ask a few native Americans wh...,OFF
864263.0,@USER She should whats a few native Americans ...,OFF
901941.0,@USER @USER Go house you’re drunk!!! @USER #MA...,OFF
168201.0,Amazon is investigating Chinese employees who ...,NOT
...,...,...
672102.0,Benidorm ♦ Creamfields ♦ Maga ♦ Not too shabby...,NOT
672103.0,Benidorm ✅ Creamfields ✅ Maga ✅ Not too shabby...,NOT
829211.0,@USER And why report this garbage. We don't ta...,OFF


In [61]:
df_real = pd.read_csv("olid-training.tsv", sep='\t', header=0, index_col='id')
df_real=df_real[['tweet','subtask_a']]
df_real

,tweet,subtask_a
id,,
86426,@USER She should ask a few native Americans wh...,OFF
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
16820,Amazon is investigating Chinese employees who ...,NOT
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT
...,...,...
95338,@USER Sometimes I get strong vibes from people...,OFF
67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT
82921,@USER And why report this garbage. We don't g...,OFF


In [62]:
# NOTE OFF should be 1, was 0 before
label_dict = {'OFF':1, 'NOT':0}

In [63]:
df_real['a_label']=df_real.subtask_a.replace(label_dict)
df_aug['a_label']=df_aug.subtask_a.replace(label_dict)
print(df_real.head())
print(df_aug.head())

                                                   tweet subtask_a  a_label
id                                                                         
86426  @USER She should ask a few native Americans wh...       OFF        1
90194  @USER @USER Go home you’re drunk!!! @USER #MAG...       OFF        1
16820  Amazon is investigating Chinese employees who ...       NOT        0
62688  @USER Someone should'veTaken" this piece of sh...       OFF        1
43605  @USER @USER Obama wanted liberals &amp; illega...       NOT        0
                                                      tweet subtask_a  a_label
id                                                                            
864261.0  @USER She should ask a few native Americans wh...       OFF        1
864262.0  @USER She should ask a few native Americans wh...       OFF        1
864263.0  @USER She should whats a few native Americans ...       OFF        1
901941.0  @USER @USER Go house you’re drunk!!! @USER #MA...       OFF    

In [64]:
!pip install emoji --upgrade

Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (1.2.0)


In [65]:
!pip install wordsegment

In [66]:
from wordsegment import load
load()

In [67]:
import re
from wordsegment import segment
import emoji
def clean(word):
  
  text = re.sub(r"can't", "can not ", word)
  text = re.sub(r"don't", "can not ", text)
  text = re.sub(r"\'s", "", word)
  text = re.sub(r"\'ve", "have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'m", "am", text)
  text = re.sub(r"%", "percent", text) 
  return text 

def foremoji(text):
  text = re.sub(r"’", "'", text)
  text = re.sub(r":", " : ", text)
  text=emoji.demojize(text)
  text = text.split()
  new_text=[]
  for w in text: 
    w=clean(w)  
    w=re.sub(r":", " <emoji> ", w)
    w=re.sub(r"@USER","",w)
    w=re.sub(r"_"," ",w)
    if re.match("^#",w):
      w=" ".join(segment(w))
    new_text.append(w)
  text = " ".join(new_text)
  return text


# def fake_foremoji(text):
#   return '<fake> '+foremoji(text)
# def real_foremoji(text):
#   return '<real> '+foremoji(text)

In [68]:
df_real['emoji_tweet'] = df_real.tweet.apply(foremoji)
df_aug['emoji_tweet'] = df_aug.tweet.apply(foremoji)

In [69]:
# df_aug['emoji_tweet']=df_aug.emoji_tweet.apply(fake_foremoji)
# df_real['emoji_tweet']=df_real.emoji_tweet.apply(real_foremoji)

In [70]:
print(df_real.head())
print(df_aug.head())

print("=" * 60)
print("Number of examples: real data --", len(df_real), "augmented data --", len(df_aug))
print("=" * 60)

                                                   tweet  ...                                        emoji_tweet
id                                                        ...                                                   
86426  @USER She should ask a few native Americans wh...  ...   She should ask a few native Americans what th...
90194  @USER @USER Go home you’re drunk!!! @USER #MAG...  ...    Go home you are drunk!!!  maga trump2020  <e...
16820  Amazon is investigating Chinese employees who ...  ...  Amazon is investigating Chinese employees who ...
62688  @USER Someone should'veTaken" this piece of sh...  ...   Someone shouldhaveTaken" this piece of shit t...
43605  @USER @USER Obama wanted liberals &amp; illega...  ...    Obama wanted liberals &amp; illegals to move...

[5 rows x 4 columns]
                                                      tweet  ...                                        emoji_tweet
id                                                           ...       

In [71]:
from sklearn.model_selection import train_test_split
RANDOM_SEED=17

X_train_real, X_val, y_train_real, y_val = train_test_split(
  df_real.index.values,
  df_real.a_label.values,
  test_size=0.2,
  random_state=RANDOM_SEED,
  # stratify=df.a_label.values
)

df_real['data_type'] = ['not_set']*df_real.shape[0]

aug_filter_index = [idx for idx in df_aug.index.values if idx // 10 not in X_val]

df_aug_filter = df_aug.loc[aug_filter_index]
df_aug_filter['data_type'] = ['train']*df_aug_filter.shape[0]
# NOTE should remove the augmented data generated from of real validation data, otherwise the validation info is leaked

df_real.loc[X_train_real, 'data_type'] = 'train'
df_real.loc[X_val, 'data_type'] = 'val'

In [72]:

df = pd.concat( [df_aug_filter, df_real], axis=0)
df

,tweet,subtask_a,a_label,emoji_tweet,data_type
id,,,,,
901941.0,@USER @USER Go house you’re drunk!!! @USER #MA...,OFF,1,Go house you are drunk!!! maga trump2020 <...,train
901941.0,@USER @USER Go house you’re drunk!!! @USER #MA...,OFF,1,Go house you are drunk!!! maga trump2020 <...,train
168201.0,Amazon is investigating Chinese employees who ...,NOT,0,Amazon is investigating Chinese employees who ...,train
168201.0,Amazon is investigating Chinese employees who ...,NOT,0,Amazon is investigating Chinese employees who ...,train
168202.0,Amazon is investigating Chinese employees who ...,NOT,0,Amazon is investigating Chinese employees who ...,train
...,...,...,...,...,...
95338.0,@USER Sometimes I get strong vibes from people...,OFF,1,Sometimes I get strong vibes from people and ...,train
67210.0,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,0,Benidorm <emoji> check mark button <emoji> C...,train
82921.0,@USER And why report this garbage. We don't g...,OFF,1,And why report this garbage. We don't give a ...,train


In [73]:
# shuffle data
df = df.sample(frac=1)
df

,tweet,subtask_a,a_label,emoji_tweet,data_type
id,,,,,
79564.0,@USER @USER You can’t tell us #Conservatives t...,NOT,0,You can't tell us conservatives that didn't...,val
606842.0,@USER You are not alone here. I didn't like Th...,NOT,0,You are not alone here. I didn't like The Las...,train
48279.0,@USER Hang in there. She is alive.,NOT,0,Hang in there. She is alive.,train
122642.0,@USER @USER @USER wow @USER are these the type...,NOT,0,wow are these the types of folks we 'll te...,train
29758.0,@USER There she is nibbling in her lower lip a...,NOT,0,There she is nibbling in her lower lip and lo...,val
...,...,...,...,...,...
95898.0,@USER Thank you she is a gentle soul xxx😘😘😘❤️❤...,NOT,0,Thank you she is a gentle soul xxx <emoji> fa...,train
479752.0,@USER @USER @USER @USER @USER @USER @USER AK h...,NOT,0,AK how can you fight hhrs when one of t...,train
12121.0,@USER @USER @USER @USER @USER @USER Yes I do! ...,NOT,0,Yes I do! A lot of us that support stric...,train


In [74]:
print(len(df[df.data_type=='train']))
print(len(df[df.data_type=='val']))

37847
2648


In [75]:
from transformers import BertTokenizer

PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [76]:

from transformers import BertForSequenceClassification

model=BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME,
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [77]:
tokenizer.add_special_tokens({'additional_special_tokens':["<emoji>"]})
model.resize_token_embeddings(len(tokenizer))

Embedding(30523, 768)

In [78]:

encoder_data_train=tokenizer.batch_encode_plus(
    df[df.data_type=='train'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

encoder_data_val=tokenizer.batch_encode_plus(
    df[df.data_type=='val'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation =True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)


In [79]:


inputs_ids_train=encoder_data_train['input_ids']
attentions_masks_train=encoder_data_train['attention_mask']
labels_train=torch.tensor(df[df.data_type=='train'].a_label.values)

inputs_ids_val=encoder_data_val['input_ids']
attentions_masks_val=encoder_data_val['attention_mask']
labels_val=torch.tensor(df[df.data_type=='val'].a_label.values)

In [80]:

from torch.utils.data import TensorDataset
dataset_train=TensorDataset(inputs_ids_train,
              attentions_masks_train, labels_train)
dataset_val=TensorDataset(inputs_ids_val,
              attentions_masks_val, labels_val)

In [81]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE=16

dataloader_train=DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=BATCH_SIZE
)
dataloader_val=DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=BATCH_SIZE
)

In [82]:

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-6, 
                  eps=1e-8)
epochs = 8
scheduler = get_linear_schedule_with_warmup(optimizer, 
                        num_warmup_steps=0,
                        num_training_steps=len(dataloader_train)*epochs)

In [83]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def classification_report_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return classification_report(labels_flat, preds_flat, digits=5)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [84]:


def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':batch[0],
              'attention_mask':batch[1],
              'labels':batch[2],
              }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [85]:
from tqdm.notebook import tqdm
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.5368490295889311
Validation loss: 0.46605484636433153
F1 Score (Weighted): 0.7785374730364159



Epoch 2
Training loss: 0.42922107955017336
Validation loss: 0.45799944379243507
F1 Score (Weighted): 0.7927417445057989



Epoch 3
Training loss: 0.3922231572688835
Validation loss: 0.4684130653110613
F1 Score (Weighted): 0.795575558517303



Epoch 4
Training loss: 0.36141412470581064
Validation loss: 0.4801512571971819
F1 Score (Weighted): 0.7922797453582836



Epoch 5
Training loss: 0.34060799117441765
Validation loss: 0.48609022885920056
F1 Score (Weighted): 0.7995902507329199



Epoch 6
Training loss: 0.3205102923785164
Validation loss: 0.5095542525670614
F1 Score (Weighted): 0.7918177562841651



Epoch 7
Training loss: 0.30891385495454854
Validation loss: 0.51327505160168
F1 Score (Weighted): 0.7918709664389296



Epoch 8
Training loss: 0.30053383870237677
Validation loss: 0.520781698894788
F1 Score (Weighted): 0.7899096378816471



In [86]:
df_sweet = pd.read_csv("testset-levela.tsv", sep='\t', header=0, index_col='id')
df_sweet.head()

,tweet
id,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
27014,"#ConstitutionDay is revered by Conservatives, ..."
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
13876,#Watching #Boomer getting the news that she is...
60133,#NoPasaran: Unity demo to oppose the far-right...


In [87]:
df_l = pd.read_csv("labels-levela.csv",header=None,names=['id','subtask_a'],index_col='id')
df_l.head()

,subtask_a
id,
15923,OFF
27014,NOT
30530,NOT
13876,NOT
60133,OFF


In [88]:
test = pd.merge(df_sweet, df_l, on=['id'])

In [89]:
label_dict = {'OFF':1, 'NOT':0}

In [90]:
test['a_label']=test.subtask_a.replace(label_dict)
test.head()

,tweet,subtask_a,a_label
id,,,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF,1
27014,"#ConstitutionDay is revered by Conservatives, ...",NOT,0
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT,0
13876,#Watching #Boomer getting the news that she is...,NOT,0
60133,#NoPasaran: Unity demo to oppose the far-right...,OFF,1


In [91]:
test['data_type']=['test']*test.shape[0]

In [92]:
test['emoji_tweet']=test.tweet.apply(foremoji)
# test['emoji_tweet']=test.emoji_tweet.apply(real_foremoji)

In [93]:
encoder_data_test=tokenizer.batch_encode_plus(
    test[test.data_type=='test'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)
inputs_ids_test=encoder_data_test['input_ids']
attentions_masks_test=encoder_data_test['attention_mask']
labels_test=torch.tensor(test[test.data_type=='test'].a_label.values)
dataset_test=TensorDataset(inputs_ids_test,
              attentions_masks_test, labels_test)
dataloader_test=DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=4
)

In [117]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_4.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [118]:
_, predictions, true_test = evaluate(dataloader_test)

In [119]:
print(classification_report_func(predictions, true_test))

              precision    recall  f1-score   support

           0    0.89389   0.89677   0.89533       620
           1    0.73109   0.72500   0.72803       240

    accuracy                        0.84884       860
   macro avg    0.81249   0.81089   0.81168       860
weighted avg    0.84846   0.84884   0.84864       860



In [120]:
!git clone https://github.com/intelligence-csd-auth-gr/Ethos-Hate-Speech-Dataset.git
!pip install -qq transformers

Cloning into 'Ethos-Hate-Speech-Dataset'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 184 (delta 101), reused 107 (delta 46), pack-reused 0
Receiving objects: 100% (184/184), 12.63 MiB | 11.94 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [121]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torchtext import data
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [122]:
data_path='Ethos-Hate-Speech-Dataset/ethos/ethos_data/Ethos_Dataset_Binary.csv'
df = pd.read_csv(data_path, delimiter=';')
df.head()

,comment,isHate
0,You should know women's sports are a joke,1.0
1,You look like Sloth with deeper Down’s syndrome,1.0
2,You look like Russian and speak like Indian. B...,1.0
3,"Women deserve to be abused, I guess.",1.0
4,Women are made for making babies and cooking d...,1.0


In [123]:
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem import SnowballStemmer

def clean_comment(text, stops=False, stemming=False):
  text = str(text)
  text = text.lower().split()
  text = " ".join(text)
  text = re.sub(r"what's", "what is ", text)
  text = re.sub(r"don't", "do not ", text)
  text = re.sub(r"aren't", "are not ", text)
  text = re.sub(r"isn't", "is not ", text)
  text = re.sub(r"%", " percent ", text)
  text = re.sub(r"that's", "that is ", text)
  text = re.sub(r"doesn't", "does not ", text)
  text = re.sub(r"he's", "he is ", text)
  text = re.sub(r"she's", "she is ", text)
  text = re.sub(r"it's", "it is ", text)
  text = re.sub(r"\'s", " ", text)
  text = re.sub(r"\'ve", " have ", text)
  text = re.sub(r"n't", " not ", text)
  text = re.sub(r"i'm", "i am ", text)
  text = re.sub(r"\'re", " are ", text)
  text = re.sub(r"\'d", " would ", text)
  text = re.sub(r"\'ll", " will ", text)
  text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
  text = re.sub(r",", " ", text)
  text = re.sub(r"\.", " ", text)
  text = re.sub(r"!", " ! ", text)
  text = re.sub(r"\/", " ", text)
  text = re.sub(r"\^", " ^ ", text)
  text = re.sub(r"\+", " + ", text)
  text = re.sub(r"\-", " - ", text)
  text = re.sub(r"\=", " = ", text)
  text = re.sub(r"'", " ", text)
  text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
  text = re.sub(r":", " : ", text)
  text = re.sub(r" u s ", " american ", text)
  text = re.sub(r"\0s", "0", text)
  text = re.sub(r" 9 11 ", "911", text)
  text = re.sub(r"e - mail", "email", text)
  text = re.sub(r"j k", "jk", text)
  text = re.sub(r"\s{2,}", " ", text)
  text = text.lower().split()
  text = [w for w in text if len(w) >= 2]
  # if stemming:
  #   englishStemmer = SnowballStemmer("english")
  #   text = [englishStemmer.stem(w) for w in text]
  # if stops:
  #   text = [w for w in text if w not in stopwords.words('english')]
  text = " ".join(text)
  return text

In [124]:
def foremoji(text):
  text = re.sub(r"’", "'", text)
  text = re.sub(r":", " : ", text)
  text=emoji.demojize(text)
  text = text.split()
  new_text=[]
  for w in text: 
    w=clean_comment(w)  
    w=re.sub(r":", " <emoji> ", w)
    w=re.sub(r"@USER","",w)
    w=re.sub(r"_"," ",w)
    if re.match("^#",w):
      w=" ".join(segment(w))
    new_text.append(w)
  text = " ".join(new_text)
  return text


def fake_foremoji(text):
  return '<fake> '+foremoji(text)
def real_foremoji(text):
  return '<real> '+foremoji(text)

In [126]:
clean_data=[]
for txt in df.comment:
  clean_data.append(txt)
df['tweet']=clean_data

In [127]:
def to_category(score):
  if score <= 0.3:
    return 'NOT'
  elif score>0.7:
    return 'OFF'

  

df['subtask_a']=df.isHate.apply(to_category)
df.head()
df.info

<bound method DataFrame.info of                                                comment  ...  subtask_a
0            You should know women's sports are a joke  ...        OFF
1      You look like Sloth with deeper Down’s syndrome  ...        OFF
2    You look like Russian and speak like Indian. B...  ...        OFF
3                 Women deserve to be abused, I guess.  ...        OFF
4    Women are made for making babies and cooking d...  ...        OFF
..                                                 ...  ...        ...
993   From the midnight sun where the hot springs blow  ...        NOT
994                        Don't say I'm not your type  ...        NOT
995   And therefore never send to know for whom the...  ...        NOT
996                      And I can't stand another day  ...        NOT
997   All values, unless otherwise stated, are in U...  ...        NOT

[998 rows x 4 columns]>

In [128]:
df.dropna(axis=0, how='any', inplace=True)

In [129]:
df.info

<bound method DataFrame.info of                                                comment  ...  subtask_a
0            You should know women's sports are a joke  ...        OFF
1      You look like Sloth with deeper Down’s syndrome  ...        OFF
2    You look like Russian and speak like Indian. B...  ...        OFF
3                 Women deserve to be abused, I guess.  ...        OFF
4    Women are made for making babies and cooking d...  ...        OFF
..                                                 ...  ...        ...
993   From the midnight sun where the hot springs blow  ...        NOT
994                        Don't say I'm not your type  ...        NOT
995   And therefore never send to know for whom the...  ...        NOT
996                      And I can't stand another day  ...        NOT
997   All values, unless otherwise stated, are in U...  ...        NOT

[768 rows x 4 columns]>

In [130]:
df['a_label']=df.subtask_a.replace(label_dict)

In [131]:

df['emoji_tweet']=df.tweet.apply(foremoji)

In [135]:
df['data_type']=['test']*df.shape[0]
encoder_data_test=tokenizer.batch_encode_plus(
    df[df.data_type=='test'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)
inputs_ids_test=encoder_data_test['input_ids']
attentions_masks_test=encoder_data_test['attention_mask']
labels_test=torch.tensor(df[df.data_type=='test'].a_label.values)
dataset_test=TensorDataset(inputs_ids_test,
              attentions_masks_test, labels_test)
dataloader_test=DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=4
)

In [138]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_3.model', map_location=torch.device('cuda')))
_, predictions, true_test = evaluate(dataloader_test)
accuracy_per_class(predictions, true_test)

Class: NOT
Accuracy: 327/482

Class: OFF
Accuracy: 243/286



In [139]:
print(classification_report_func(predictions, true_test))

              precision    recall  f1-score   support

           0    0.88378   0.67842   0.76761       482
           1    0.61055   0.84965   0.71053       286

    accuracy                        0.74219       768
   macro avg    0.74717   0.76404   0.73907       768
weighted avg    0.78203   0.74219   0.74635       768

